In [75]:
import sys
sys.path.append('/home/wangs/rs/lib')
import ff
import pickle
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_style("whitegrid")
sns.set_palette("pastel")
from multiprocessing import Pool
from tqdm import tqdm
import statsmodels.api as sm
from datetime import datetime
import mplfinance as mpf
from functools import partial

In [76]:
stock_index = ff.read('一字板').index
time_index = ff.read('一字板').columns
up = ff.read('up').loc[stock_index,time_index].copy()
open = ff.read('open').loc[stock_index,time_index].copy()
close = ff.read('close').loc[stock_index,time_index].copy()
high = ff.read('high').loc[stock_index,time_index].copy()
low = ff.read('low').loc[stock_index,time_index].copy()
up_limit = pd.DataFrame(ff.read('up_limit').copy(),index = stock_index,columns = time_index) * ff.filter0.loc[stock_index,time_index]

In [77]:
up_filter = (high == up_limit)
dict = {}
for i in up_filter.sum(axis = 1)[up_filter.sum(axis = 1)>0].index:
    stock = up_filter.loc[i,:]
    dict[i] = list(stock[stock == True].index)

In [163]:
def up_first(stock):
    first = []
    for time in dict[stock]:
        start_time = pd.to_datetime(time).strftime('%Y-%m-%d') + ' 09:31:00'
        end_time = pd.to_datetime(time).strftime('%Y-%m-%d') + ' 15:00:00'
        data = ff.read_min(stock).loc[start_time:end_time,:]
        daily_up_limit = data.loc[:,'high_limit']
        judge = (data.loc[:,'close'] == daily_up_limit) * (data.loc[:,'high'] == daily_up_limit) * (data.loc[:,'low'] == daily_up_limit)
        try:
            start_1 = np.argwhere((judge).values)[0][0]
            first.append(start_1)
        except:
            first.append(np.nan)
            pass
    return first

In [164]:
with Pool(24) as p:
    res_lst = list(tqdm(p.imap(up_first,dict.keys()),total = len(dict.keys())))
up_first_factor = pd.DataFrame(np.nan,index = up.index,columns = up.columns)
for i in range(len(res_lst)):
    for j in range(len(res_lst[i])):
        up_first_factor.loc[list(dict.keys())[i],list(dict.values())[i][j]] = res_lst[i][j]

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4417/4417 [04:02<00:00, 18.20it/s]


In [178]:
up_first_factor.count()

20200825     36
20200826     26
20200827     37
20200828     45
20200831     68
           ... 
20240228     83
20240229    121
20240301     83
20240304     87
20240305     59
Length: 854, dtype: int64

In [144]:
dict['000056.SZ']

['20201026',
 '20210224',
 '20210225',
 '20210226',
 '20210301',
 '20210407',
 '20210408',
 '20210414',
 '20210514',
 '20210520',
 '20210527',
 '20210603',
 '20210719',
 '20210802',
 '20210803',
 '20211021',
 '20211115',
 '20211229',
 '20211230',
 '20211231',
 '20220104',
 '20220105',
 '20220112',
 '20220114',
 '20220208',
 '20220209',
 '20220225',
 '20220323',
 '20220421',
 '20220506',
 '20220509',
 '20220519',
 '20220721',
 '20220722',
 '20220725',
 '20220726',
 '20220727',
 '20220805',
 '20220808',
 '20220812',
 '20220913',
 '20221115',
 '20221116',
 '20230111',
 '20230118',
 '20230206',
 '20230427',
 '20230428',
 '20230615',
 '20230721',
 '20230907',
 '20231107',
 '20231108',
 '20231109',
 '20231110',
 '20231113',
 '20231114',
 '20231115',
 '20231121',
 '20240115',
 '20240219',
 '20240221',
 '20240222',
 '20240226',
 '20240227',
 '20240305']

In [162]:
up_first('000056.SZ')

[239,
 12,
 1,
 0,
 nan,
 217,
 nan,
 nan,
 174,
 nan,
 nan,
 nan,
 10,
 33,
 14,
 13,
 141,
 23,
 10,
 10,
 109,
 15,
 191,
 68,
 199,
 39,
 163,
 nan,
 68,
 6,
 27,
 7,
 52,
 14,
 13,
 1,
 nan,
 9,
 4,
 47,
 nan,
 96,
 nan,
 7,
 1,
 nan,
 2,
 0,
 134,
 12,
 nan,
 17,
 0,
 0,
 0,
 0,
 3,
 0,
 nan,
 44,
 158,
 125,
 nan,
 59,
 129,
 153]

In [159]:
first = []
time = '20210301'
stock = '000056.SZ'
start_time = pd.to_datetime(time).strftime('%Y-%m-%d') + ' 09:31:00'
end_time = pd.to_datetime(time).strftime('%Y-%m-%d') + ' 15:00:00'
data = ff.read_min(stock).loc[start_time:end_time,:]
daily_up_limit = data.loc[:,'high_limit']
daily_up_limit
judge = (data.loc[:,'close'] == daily_up_limit) * (data.loc[:,'high'] == daily_up_limit) * (data.loc[:,'low'] == daily_up_limit)
try:
    start_1 = np.argwhere((judge).values)[0][0]
    first.append(start_1)
except:
    first.append(np.nan)
    pass
first

[nan]

In [126]:
len(res_lst[37]

52

In [85]:
up_filter['20240305'].replace({False:np.nan,True:1.0}).dropna()

000056.SZ    1.0
000506.SZ    1.0
000550.SZ    1.0
000576.SZ    1.0
000586.SZ    1.0
            ... 
603773.SH    1.0
603789.SH    1.0
603863.SH    1.0
605178.SH    1.0
605555.SH    1.0
Name: 20240305, Length: 74, dtype: float64

In [86]:
dict['000056.SZ']

['20201026',
 '20210224',
 '20210225',
 '20210226',
 '20210301',
 '20210407',
 '20210408',
 '20210414',
 '20210514',
 '20210520',
 '20210527',
 '20210603',
 '20210719',
 '20210802',
 '20210803',
 '20211021',
 '20211115',
 '20211229',
 '20211230',
 '20211231',
 '20220104',
 '20220105',
 '20220112',
 '20220114',
 '20220208',
 '20220209',
 '20220225',
 '20220323',
 '20220421',
 '20220506',
 '20220509',
 '20220519',
 '20220721',
 '20220722',
 '20220725',
 '20220726',
 '20220727',
 '20220805',
 '20220808',
 '20220812',
 '20220913',
 '20221115',
 '20221116',
 '20230111',
 '20230118',
 '20230206',
 '20230427',
 '20230428',
 '20230615',
 '20230721',
 '20230907',
 '20231107',
 '20231108',
 '20231109',
 '20231110',
 '20231113',
 '20231114',
 '20231115',
 '20231121',
 '20240115',
 '20240219',
 '20240221',
 '20240222',
 '20240226',
 '20240227',
 '20240305']

In [88]:
up_first_factor['20240305'].dropna()

301213.SZ     6.0
600584.SH     0.0
600774.SH    15.0
Name: 20240305, dtype: float64

In [130]:
for i in range(len(res_lst)):
    for j in range(len(res_lst[i])):
        if list(dict.keys())[i] == '000056.SZ':
            print(list(dict.values())[i][j])
        up_first_factor.loc[list(dict.keys())[i],list(dict.values())[i][j]] = res_lst[i][j]

20201026
20210224
20210225
20210226
20210301
20210407
20210408
20210414
20210514
20210520
20210527
20210603
20210719
20210802
20210803
20211021
20211115
20211229
20211230
20211231
20220104
20220105
20220112
20220114
20220208
20220209
20220225
20220323
20220421
20220506
20220509
20220519
20220721
20220722
20220725
20220726
20220727
20220805
20220808
20220812
20220913
20221115
20221116
20230111
20230118
20230206
20230427
20230428
20230615
20230721
20230907
20231107


In [103]:
up_first_factor.loc['000056.SZ'].dropna()

20201026    239.0
20210224     12.0
20210225      1.0
20210226      0.0
20210301    217.0
20210407    174.0
20210408     10.0
20210414     33.0
20210514     14.0
20210520     13.0
20210527    141.0
20210603     23.0
20210719     10.0
20210802     10.0
20210803    109.0
20211021     15.0
20211115    191.0
20211229     68.0
20211230    199.0
20211231     39.0
20220104    163.0
20220105     68.0
20220112      6.0
20220114     27.0
20220208      7.0
20220209     52.0
20220225     14.0
20220323     13.0
20220421      1.0
20220506      9.0
20220509      4.0
20220519     47.0
20220721     96.0
20220722      7.0
20220725      1.0
20220726      2.0
20220727      0.0
20220805    134.0
20220808     12.0
20220812     17.0
20220913      0.0
20221115      0.0
20221116      0.0
20230111      0.0
20230118      3.0
20230206      0.0
20230427     44.0
20230428    158.0
20230615    125.0
20230721     59.0
20230907    129.0
20231107    153.0
Name: 000056.SZ, dtype: float64

In [106]:
up_filter['20240305'].replace({False:np.nan,True:1.0}).dropna()

000056.SZ    1.0
000506.SZ    1.0
000550.SZ    1.0
000576.SZ    1.0
000586.SZ    1.0
            ... 
603773.SH    1.0
603789.SH    1.0
603863.SH    1.0
605178.SH    1.0
605555.SH    1.0
Name: 20240305, Length: 74, dtype: float64

In [194]:
ff.read_min('603256.SH')['2020-08-25 09:31:00':'2020-08-25 15:00:00'].iloc[200:300]

,open,close,low,high,volume,money,high_limit,low_limit,paused
2020-08-25 14:21:00,12.97,12.97,12.96,12.97,8400.0,108927.0,14.14,11.57,0.0
2020-08-25 14:22:00,12.97,12.97,12.96,12.97,6900.0,89490.0,14.14,11.57,0.0
2020-08-25 14:23:00,12.96,12.97,12.96,12.98,13700.0,177585.0,14.14,11.57,0.0
2020-08-25 14:24:00,12.96,12.95,12.95,12.97,12500.0,161933.0,14.14,11.57,0.0
2020-08-25 14:25:00,12.95,12.94,12.94,12.95,8500.0,110014.0,14.14,11.57,0.0
2020-08-25 14:26:00,12.94,12.94,12.94,12.94,7800.0,100932.0,14.14,11.57,0.0
2020-08-25 14:27:00,12.95,12.96,12.95,12.96,8800.0,114011.0,14.14,11.57,0.0
2020-08-25 14:28:00,12.96,12.96,12.96,12.97,7100.0,92017.0,14.14,11.57,0.0
2020-08-25 14:29:00,12.96,12.96,12.95,12.97,4800.0,62212.0,14.14,11.57,0.0
2020-08-25 14:30:00,12.96,12.95,12.95,12.96,5500.0,71258.0,14.14,11.57,0.0


In [185]:
ff.read('第一次封板时刻')['20200825'].dropna()

000881.SZ     52.0
000897.SZ     10.0
002327.SZ     32.0
002429.SZ     31.0
002534.SZ      0.0
002537.SZ     19.0
002677.SZ      7.0
002855.SZ      1.0
002861.SZ      2.0
002885.SZ      1.0
002903.SZ     83.0
300106.SZ     34.0
300116.SZ     24.0
300152.SZ    132.0
300176.SZ    100.0
300269.SZ     28.0
300313.SZ     21.0
300368.SZ    219.0
300446.SZ     73.0
600158.SH      1.0
600189.SH    229.0
600396.SH      1.0
600976.SH     39.0
603076.SH     98.0
603161.SH     55.0
603167.SH     11.0
603226.SH      7.0
603535.SH    139.0
603595.SH      0.0
603663.SH     93.0
603683.SH     23.0
603688.SH     44.0
603709.SH    219.0
603787.SH    165.0
603826.SH     34.0
603848.SH     52.0
Name: 20200825, dtype: float64

In [186]:
up_filter.replace({False:np.nan,True:1.0})['20200825'].dropna()

000881.SZ    1.0
000897.SZ    1.0
002327.SZ    1.0
002429.SZ    1.0
002534.SZ    1.0
002537.SZ    1.0
002677.SZ    1.0
002855.SZ    1.0
002861.SZ    1.0
002885.SZ    1.0
002903.SZ    1.0
300106.SZ    1.0
300116.SZ    1.0
300135.SZ    1.0
300152.SZ    1.0
300176.SZ    1.0
300269.SZ    1.0
300313.SZ    1.0
300368.SZ    1.0
300446.SZ    1.0
300592.SZ    1.0
600158.SH    1.0
600189.SH    1.0
600396.SH    1.0
600543.SH    1.0
600726.SH    1.0
600976.SH    1.0
603076.SH    1.0
603161.SH    1.0
603167.SH    1.0
603226.SH    1.0
603256.SH    1.0
603535.SH    1.0
603595.SH    1.0
603663.SH    1.0
603683.SH    1.0
603688.SH    1.0
603709.SH    1.0
603787.SH    1.0
603826.SH    1.0
603848.SH    1.0
Name: 20200825, dtype: float64

In [198]:
stock_list = ['000001.SZ','000002.SZ']

In [249]:
ff.read('close').loc['002348.SZ','20240626']

2.4

In [243]:
30.71/31.16

0.9855584082156611

In [246]:
1-0.853298/0.900040

0.05193324741122607

In [247]:
1-0.853298

0.146702

In [250]:
ff.read('板上成交量占比').count()

20200825     36
20200826     26
20200827     37
20200828     45
20200831     68
           ... 
20240228     83
20240229    121
20240301     83
20240304     87
20240305     59
Length: 854, dtype: int64